In [28]:
import os
os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/processed_data/all_cities_combined_clean.csv")

In [5]:
df

,id,city,section,subsection,text
0,0,Cairo,See,Further afield,29.8 31.233333 1 Dahshur Pyramids . For a cont...
1,1,Cairo,See,NaN,29.8 31.233333 5 Dahshur Pyramids . For a cont...
2,2,Cairo,See,NaN,30.0405 31.2646 2 Al-Azhar Park . A landscaped...
3,3,Cairo,See,NaN,30.0417 31.2483 4 Abdeen Palace . About 1 km e...
4,4,Cairo,See,NaN,30.0475 31.262222 3 Khan El Khalily . Cairo's ...
...,...,...,...,...,...
539,539,Seoul,Districts,NaN,"West ( Gangseo-gu, Yangcheon-gu ) Western area..."
540,540,Seoul,See,NaN,While Seoul today is mostly known as a super-m...
541,541,Seoul,Districts,NaN,Yeongdeungpo & Guro (영등포 / 구로) ( Yeongdeungpo-...
542,542,Seoul,Districts,NaN,Yongsan (용산) ( Yongsan-gu ) Yongsan is home to...


In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py --no-check-certificate

--2025-09-23 11:07:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273 (4.2K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   4.17K  --.-KB/s    in 0.002s  

2025-09-23 11:07:29 (2.65 MB/s) - ‘minsearch.py’ saved [4273/4273]



In [8]:
import minsearch

/mnt/d/Travel Assistant/Musafir/notebooks/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [9]:
df.columns

Index(['id', 'city', 'section', 'subsection', 'text'], dtype='object')

In [10]:
documents = df.to_dict(orient='records')

In [11]:
documents[15]

{'id': 15,
 'city': 'Cairo',
 'section': 'Eat',
 'subsection': 'Self-catering',
 'text': 'By far the cheapest and most satisfying option, buying from souks and outdoor markets makes for a crash course in Arabic and haggling, not to mention that the produce is often superb! Bread can be found on nearly every corner and comes in two types - whole wheat aysh baladi and white flour aysh shami . Both are baked fresh daily and delivered by thousands of kids on bicycles to every corner of the city. Every neighborhood has a few streets dedicated to produce and other goods. Always wash fruit thoroughly before eating. Eating a fresh Roma tomato in the heat of Summer straight from a market seller after being washed is a delight, hard to match. The fruits and vegetables in Egypt may not conform to EU or US standards of size, but their taste is far superior.'}

In [18]:
index = minsearch.Index(
    text_fields=['city', 'section', 'text'],
    keyword_fields=['id']
)

In [19]:
index.fit(documents)

In [20]:
query = "give me a three-day trip to London"

In [21]:
index.search(query, num_results=10)

[{'id': 317,
  'city': 'London',
  'section': 'See',
  'subsection': 'London Pass',
  'text': 'Whereas some London museums offer free entry, some other top London attractions are ridiculously expensive. For example, entry to Westminster Abbey costs £20 per person (adult), and entry to the Tower is £21.50 per adult if bought online (2017). These prices can be sometimes mitigated by a purchase of London Pass, which needs to be done at the London Pass website . The pass comes in several varieties and gives access to over 60 attractions, including both Westminster Abbey and the Tower. For example, a day pass costs £62 for an adult (2017). The best strategy, if one wants to visit several expensive high-profile attractions, is to buy a day pass and to try visiting all of them in the same day. This requires some advanced planning and will not give you much time at each place you visit - for example, it can take an hour on public transport to travel between the Tower of London and London Zoo.'

In [23]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv

In [24]:
# loads variables from .env
load_dotenv()  

True

In [25]:
api_key = os.getenv("API_KEY")

In [29]:
client = Mistral(api_key = api_key)
response = client.chat.complete(
    model= "ministral-8b-latest",
    messages=[UserMessage(content=query)],
)

print(response.choices[0].message.content)

**Three-Day Trip to London**

**Day 1: Iconic Landmarks and History**

**Morning:**
- Start your day at **Buckingham Palace** to witness the Changing of the Guard ceremony (check the schedule in advance).
- Walk to **Westminster Abbey** and explore the historic church where many British monarchs are buried.

**Afternoon:**
- Head to the **Houses of Parliament** and **Big Ben** (currently under renovation, but still worth a visit).
- Cross the **Westminster Bridge** to reach the **London Eye** for a panoramic view of the city.

**Evening:**
- Enjoy dinner at a traditional British pub, such as **The Lamb & Flag** in Covent Garden.
- Take a leisurely stroll through **Covent Garden** and enjoy the street performers.

**Day 2: Museums and Culture**

**Morning:**
- Visit the **British Museum** to explore its vast collection of artifacts from around the world.
- Walk to **Trafalgar Square** and see Nelson's Column and the National Gallery.

**Afternoon:**
- Head to **The Victoria and Albert M

In [30]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [34]:
prompt_template = """
You're a travel assistant. Answer the QUESTION based on the CONTEXT from our travel database.
Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: {question}
CONTEXT:
{context}
""".strip()

entry_template = """
 City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [35]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [36]:
print(prompt)

You're a travel assistant. Answer the QUESTION based on the CONTEXT from our travel database.
Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: give me a three-day trip to London
CONTEXT:
City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'

City: 'city',
 Section: 'section',
 Subsection: 'subsection',
 Text:'text'


In [37]:
def llm(prompt, model='ministral-8b-latest'):
    client = Mistral(api_key = api_key)
    response = client.chat.complete(
        model= model,
        messages=[UserMessage(content=prompt)],
    )    
    return response.choices[0].message.content

In [38]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
answer = rag(query)
print(answer)

Based on the provided context, here's a suggested three-day trip to London:

**Day 1: Explore Central London**
- Morning: Visit the British Museum to see the Rosetta Stone and other ancient artifacts.
- Afternoon: Walk along the River Thames and visit the Houses of Parliament and Big Ben.
- Evening: Explore Covent Garden and enjoy a West End show.

**Day 2: Historic Sites and Museums**
- Morning: Visit the Tower of London and see the Crown Jewels.
- Afternoon: Head to the Victoria and Albert Museum to explore art and design.
- Evening: Take a stroll through Hyde Park and visit the Serpentine Lake.

**Day 3: Cultural London**
- Morning: Visit the National Gallery to see works by famous artists like Van Gogh and Monet.
- Afternoon: Explore Camden Market for unique shopping and street food.
- Evening: Take a cruise on the River Thames to see London's landmarks from the water.


In [40]:
# Ask another question 
answer = rag('I want some tips before visting cairo')
print(answer)

Based on the provided context, here are some tips for visiting Cairo:

1. **Best Time to Visit**: The best time to visit Cairo is during the fall (September to November) or spring (March to May) when the weather is pleasant and temperatures are mild.

2. **Dress Code**: Dress modestly, especially when visiting religious sites. Women should cover their shoulders and knees, and both men and women should avoid wearing shorts or sleeveless tops.

3. **Safety**: Cairo is generally safe, but pickpocketing can occur in crowded areas. Keep your belongings secure and be cautious in tourist areas.

4. **Transportation**: Taxis are widely available and relatively inexpensive. However, it's recommended to use licensed taxis or ride-hailing services like Uber or Careem for safety and convenience.

5. **Must-See Attractions**:
   - The Great Pyramids of Giza
   - The Egyptian Museum
   - The Citadel of Saladin
   - Khan el-Khalili Bazaar
   - The Hanging Church (Church of St. Sergius)

6. **Food**: 